# Green Bond Tracker - Demo Notebook

This notebook demonstrates how to load, validate, and visualize green bond data with geographic information.

**Educational Notice:** This project is for educational and analytical purposes only. It is not intended for investment advice or financial decision-making.

## 1. Setup and Imports

In [ ]:
import sysfrom pathlib import Path# Add parent directory to path to import src modulesys.path.insert(0, str(Path.cwd().parent))import pandas as pdimport geopandas as gpdimport matplotlib.pyplot as pltfrom src.data_loader import (    load_green_bonds,    load_country_geometries,    validate_bond_data,    join_bonds_with_geo,    get_summary_statistics)from src.visuals import (    create_project_type_bar_chart,    create_choropleth_map,    save_figure,    create_and_save_all_visuals)# Set display optionspd.set_option('display.max_columns', None)pd.set_option('display.width', None)pd.set_option('display.max_colwidth', 50)

## 2. Load and Validate Data

In [ ]:
# Load green bond data
bonds = load_green_bonds()
print(f"Loaded {len(bonds)} green bonds")
print("\nFirst few records:")
bonds.head()

In [ ]:
# Validate the data
is_valid, issues = validate_bond_data(bonds)
if is_valid:
    print("✓ Data validation passed!")
else:
    print("⚠ Data validation found issues:")
    for issue in issues:
        print(f"  - {issue}")

In [ ]:
# Load country geometries
countries = load_country_geometries()
print(f"Loaded geometries for {len(countries)} countries")
print("\nCountries included:")
countries[['name', 'iso_a3', 'continent']].head(10)

## 3. Summary Statistics

In [ ]:
# Get summary statistics
stats = get_summary_statistics(bonds)
print("Green Bond Portfolio Summary")
print("=" * 60)
print(f"Total Bonds: {stats['total_bonds']}")
print(f"Total Amount: ${stats['total_amount_usd_millions']:,.2f} million USD")
print(f"Average Bond Size: ${stats['average_bond_size_usd_millions']:,.2f} million USD")
print(f"Median Bond Size: ${stats['median_bond_size_usd_millions']:,.2f} million USD")
print(f"Unique Issuers: {stats['unique_issuers']}")
print(f"Unique Countries: {stats['unique_countries']}")

if 'earliest_issue' in stats:
    print(f"\nDate Range: {stats['earliest_issue'].strftime('%Y-%m-%d')} to {stats['latest_issue'].strftime('%Y-%m-%d')}")

print("\nTop 5 Countries by Total Amount:")
for country, amount in stats['top_5_countries'].items():
    print(f"  {country}: ${amount:,.2f} million USD")

## 4. Use of Proceeds Analysis

In [ ]:
# Analyze use of proceeds
proceeds_summary = bonds.groupby('use_of_proceeds').agg({
    'amount_usd_millions': ['sum', 'count']
}).round(2)
proceeds_summary.columns = ['Total Amount (USD millions)', 'Number of Bonds']
proceeds_summary = proceeds_summary.sort_values('Total Amount (USD millions)', ascending=False)

print("\nGreen Bond Use of Proceeds:")
print(proceeds_summary)

In [ ]:
# Visualize use of proceeds
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart by amount
proceeds_by_amount = bonds.groupby('use_of_proceeds')['amount_usd_millions'].sum()
ax1.pie(proceeds_by_amount.values, labels=proceeds_by_amount.index, autopct='%1.1f%%', startangle=90)
ax1.set_title('Green Bond Allocation by Amount', fontsize=14, fontweight='bold')

# Bar chart by count
proceeds_by_count = bonds.groupby('use_of_proceeds').size()
proceeds_by_count.sort_values(ascending=True).plot(kind='barh', ax=ax2, color='green', alpha=0.7)
ax2.set_xlabel('Number of Bonds')
ax2.set_title('Number of Bonds by Use of Proceeds', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('../maps/use_of_proceeds_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 4.5. Project Type Distribution - Enhanced Visualization

Using the new `visuals.py` module, we can create publication-quality charts with just a few lines of code.

In [ ]:
# Create a bar chart of project types using the visuals module
fig = create_project_type_bar_chart(bonds, figsize=(14, 7))
plt.show()

# Optionally save the chart
# save_figure(fig, 'project_type_distribution.png')

## 5. Geographic Visualization - Choropleth Map

In [ ]:
# Join bond data with country geometries
geo_bonds = join_bonds_with_geo(bonds, countries)
print(f"Created geographic dataset with {len(geo_bonds)} countries")
print("\nCountries with green bonds:")
geo_bonds[geo_bonds['bond_count'] > 0][['name', 'bond_count', 'total_amount_usd_millions']].sort_values(
    'total_amount_usd_millions', ascending=False
)

In [ ]:
# Create choropleth map - Total Amount
fig, ax = plt.subplots(1, 1, figsize=(16, 10))

# Plot countries with bond data
geo_bonds.plot(
    column='total_amount_usd_millions',
    ax=ax,
    legend=True,
    cmap='YlGn',
    edgecolor='black',
    linewidth=0.5,
    legend_kwds={
        'label': 'Total Green Bond Amount (USD millions)',
        'orientation': 'horizontal',
        'shrink': 0.6,
        'pad': 0.05
    },
    missing_kwds={'color': 'lightgrey', 'label': 'No Data'}
)

# Add country labels for countries with bonds
for idx, row in geo_bonds[geo_bonds['bond_count'] > 0].iterrows():
    centroid = row['geometry'].centroid
    ax.annotate(
        text=row['iso_a3'],
        xy=(centroid.x, centroid.y),
        ha='center',
        fontsize=8,
        color='darkblue',
        weight='bold'
    )

ax.set_title('Green Bond Issuance by Country - Total Amount', fontsize=16, fontweight='bold', pad=20)
ax.axis('off')
plt.tight_layout()
plt.savefig('../maps/green_bonds_choropleth_amount.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create choropleth map - Number of Bonds
fig, ax = plt.subplots(1, 1, figsize=(16, 10))

geo_bonds.plot(
    column='bond_count',
    ax=ax,
    legend=True,
    cmap='Blues',
    edgecolor='black',
    linewidth=0.5,
    legend_kwds={
        'label': 'Number of Green Bonds',
        'orientation': 'horizontal',
        'shrink': 0.6,
        'pad': 0.05
    },
    missing_kwds={'color': 'lightgrey', 'label': 'No Data'}
)

# Add country labels for countries with bonds
for idx, row in geo_bonds[geo_bonds['bond_count'] > 0].iterrows():
    centroid = row['geometry'].centroid
    ax.annotate(
        text=f"{row['iso_a3']}\n({int(row['bond_count'])})",
        xy=(centroid.x, centroid.y),
        ha='center',
        fontsize=7,
        color='darkblue',
        weight='bold'
    )

ax.set_title('Green Bond Issuance by Country - Number of Bonds', fontsize=16, fontweight='bold', pad=20)
ax.axis('off')
plt.tight_layout()
plt.savefig('../maps/green_bonds_choropleth_count.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Time Series Analysis

In [ ]:
# Analyze issuance over time
bonds_with_dates = bonds.dropna(subset=['issue_date'])
bonds_with_dates['year_month'] = bonds_with_dates['issue_date'].dt.to_period('M')

# Monthly issuance
monthly_issuance = bonds_with_dates.groupby('year_month').agg({
    'amount_usd_millions': 'sum',
    'bond_id': 'count'
}).reset_index()
monthly_issuance.columns = ['Month', 'Total Amount', 'Number of Bonds']

# Convert period to timestamp for plotting
monthly_issuance['Month'] = monthly_issuance['Month'].dt.to_timestamp()

# Plot time series
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Amount over time
ax1.plot(monthly_issuance['Month'], monthly_issuance['Total Amount'], 
         marker='o', linewidth=2, markersize=6, color='green')
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Total Amount (USD millions)', fontsize=12)
ax1.set_title('Green Bond Issuance Amount Over Time', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Count over time
ax2.bar(monthly_issuance['Month'], monthly_issuance['Number of Bonds'], 
        color='steelblue', alpha=0.7, width=20)
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Number of Bonds', fontsize=12)
ax2.set_title('Number of Green Bonds Issued Over Time', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../maps/time_series_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Certification Analysis

In [ ]:
# Analyze by certification
cert_summary = bonds.groupby('certification').agg({
    'amount_usd_millions': ['sum', 'count', 'mean']
}).round(2)
cert_summary.columns = ['Total Amount (USD millions)', 'Number of Bonds', 'Average Size (USD millions)']

print("\nCertification Summary:")
print(cert_summary)

# Visualize certification distribution
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
cert_summary['Total Amount (USD millions)'].plot(kind='bar', ax=ax, color=['#2ecc71', '#3498db'], alpha=0.8)
ax.set_xlabel('Certification', fontsize=12)
ax.set_ylabel('Total Amount (USD millions)', fontsize=12)
ax.set_title('Green Bond Amount by Certification Standard', fontsize=14, fontweight='bold')
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../maps/certification_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Generate All Visualizations

Create and save all standard visualizations at once using the convenience function.

In [ ]:
# Generate all visualizations and save to outputs/ directory
saved_files = create_and_save_all_visuals(bonds, geo_bonds)

print("\nGenerated files:")
for name, filepath in saved_files.items():
    print(f"  ✓ {name}: {filepath.name}")

## Summary

This notebook demonstrated:
1. Loading and validating green bond data
2. Loading geographic data with ISO country codes
3. Calculating summary statistics
4. Analyzing use of proceeds
5. Creating choropleth maps to visualize geographic distribution
6. Time series analysis of bond issuance
7. Certification standard analysis

**Important Disclaimer:** This analysis is for educational purposes only and should not be used as the basis for investment decisions. Always consult with qualified financial advisors before making investment decisions.

## Next Steps

This demo provides a foundation for green bond tracking and analysis. Here are some planned enhancements to extend the project:

### Planned Features

1. **Interactive Mapping**
   - Integrate Folium or Plotly for interactive choropleth maps
   - Add hover tooltips showing bond details by country
   - Enable zoom and pan functionality for detailed exploration

2. **Expanded Taxonomies**
   - Add EU Taxonomy alignment indicators
   - Include SDG (Sustainable Development Goals) mapping
   - Track alignment with Paris Agreement goals
   - Add sector classification (energy, transport, buildings, etc.)

3. **Enhanced Data**
   - Expand sample dataset to 100+ bonds
   - Add more countries and regions
   - Include impact metrics (CO2 avoided, renewable capacity, etc.)
   - Track bond ratings and credit quality

4. **Advanced Analytics**
   - Yield curve analysis for green bonds
   - Greenium calculation (pricing difference vs conventional bonds)
   - Trend analysis and forecasting
   - Portfolio optimization tools

5. **Data Integration**
   - API connectors for real-time bond data
   - Integration with ESG data providers
   - Automated data refresh workflows

6. **Reporting Tools**
   - Generate PDF/HTML reports
   - Export visualizations in multiple formats
   - Create dashboard templates

### Community Contributions Welcome

This is an open educational project. Contributions are welcome in any of these areas:
- Data quality improvements
- New visualization types
- Additional analysis methods
- Documentation enhancements
- Bug fixes and optimizations

Feel free to submit pull requests or open issues on GitHub!